# Imports

In [350]:
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding

# Model Params

In [351]:
np.random.seed(42)
initial_params = np.random.random([7])
variety_to_ignore = 3

steps = 6000
step_size = 0.3

# Import dataset

In [352]:
columns = ['sepal.length', 'sepal.width', 'petal.length', 'petal.width']
dataset = pd.read_csv('iris.csv', usecols=columns)
dataset_class = pd.read_csv('iris.csv', usecols=['variety'])

In [353]:
dataset = dataset.drop(dataset[dataset_class.variety == variety_to_ignore].index)
print(dataset)
dataset = dataset.values.tolist()
features = dataset[0]

    sepal.length  sepal.width  petal.length  petal.width
0            5.1          3.5           1.4          0.2
1            4.9          3.0           1.4          0.2
2            4.7          3.2           1.3          0.2
3            4.6          3.1           1.5          0.2
4            5.0          3.6           1.4          0.2
..           ...          ...           ...          ...
95           5.7          3.0           4.2          1.2
96           5.7          2.9           4.2          1.3
97           6.2          2.9           4.3          1.3
98           5.1          2.5           3.0          1.1
99           5.7          2.8           4.1          1.3

[100 rows x 4 columns]


In [354]:
dataset_class = dataset_class.apply(lambda x: x/x.max(), axis=0)
dataset_class = dataset_class.drop(dataset_class[dataset_class.variety == variety_to_ignore/3].index)
print(dataset_class)

     variety
0   0.333333
1   0.333333
2   0.333333
3   0.333333
4   0.333333
..       ...
95  0.666667
96  0.666667
97  0.666667
98  0.666667
99  0.666667

[100 rows x 1 columns]


In [355]:
# dataset = dataset.apply(lambda x: x/x.max(), axis=0)
# dataset = dataset.join(dataset_class)
# print(dataset)

# Circuit creation

In [356]:
device = qml.device("default.qubit", wires=4)

In [357]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    AmplitudeEmbedding(features=features, wires=range(4), normalize=True, pad_with=0.)

    # First layer
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.RY(params[2], wires=2)
    qml.RY(params[3], wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[3, 2])

    # Second layer
    qml.RY(params[4], wires=1)
    qml.RY(params[5], wires=2)
    qml.CNOT(wires=[1, 2])

    # Third layer
    qml.RY(params[6], wires=2)

    # Measurement
    return qml.expval(qml.PauliZ(2))

In [358]:
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361]

Example features: [5.1, 3.5, 1.4, 0.2]

Expectation value: -0.20621869264768833

 0: ──╭QubitStateVector(M0)──RY(0.375)──╭C─────────────────────────────┤     
 1: ──├QubitStateVector(M0)──RY(0.951)──╰X──RY(0.156)──╭C──────────────┤     
 2: ──├QubitStateVector(M0)──RY(0.732)──╭X──RY(0.156)──╰X──RY(0.0581)──┤ ⟨Z⟩ 
 3: ──╰QubitStateVector(M0)──RY(0.599)──╰C─────────────────────────────┤     
M0 =
[0.80377277+0.j 0.55160877+0.j 0.22064351+0.j 0.0315205 +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j]



# Training

In [359]:
# def cost(params):
#     circuit_params = params.get('circuit_params')
#     features = params.get('features')
#     expected_value = params.get('expected_class')

#     value = circuit(features, params)

#     return ((expected_value - value) ** 2)/steps

In [360]:
params = initial_params
opt = qml.GradientDescentOptimizer(stepsize=step_size)

for i in range(steps):
    features = dataset[i%100]
    expected_value = dataset_class['variety'][i%100]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(dataset)

    params = opt.step(cost, params)

#     print(f'step: {i}')
#     print(f'features: {features}')
#     print(f'expected_value: {expected_value}')
#     print(f'value: {value}')

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

Cost after step     5:  0.0028310
Cost after step    10:  0.0025744
Cost after step    15:  0.0022892
Cost after step    20:  0.0023532
Cost after step    25:  0.0023683
Cost after step    30:  0.0020747
Cost after step    35:  0.0017969
Cost after step    40:  0.0017391
Cost after step    45:  0.0017449
Cost after step    50:  0.0014977
Cost after step    55:  0.0045544
Cost after step    60:  0.0042936
Cost after step    65:  0.0038910
Cost after step    70:  0.0038909
Cost after step    75:  0.0035532
Cost after step    80:  0.0032817
Cost after step    85:  0.0033620
Cost after step    90:  0.0029742
Cost after step    95:  0.0028894
Cost after step   100:  0.0026570
Cost after step   105:  0.0004481
Cost after step   110:  0.0003853
Cost after step   115:  0.0003812
Cost after step   120:  0.0003661
Cost after step   125:  0.0003637
Cost after step   130:  0.0003110
Cost after step   135:  0.0002575
Cost after step   140:  0.0002651
Cost after step   145:  0.0002561
Cost after ste

Cost after step  1215:  0.0000636
Cost after step  1220:  0.0001172
Cost after step  1225:  0.0001625
Cost after step  1230:  0.0001452
Cost after step  1235:  0.0001432
Cost after step  1240:  0.0001114
Cost after step  1245:  0.0001517
Cost after step  1250:  0.0000968
Cost after step  1255:  0.0001420
Cost after step  1260:  0.0001561
Cost after step  1265:  0.0001174
Cost after step  1270:  0.0001344
Cost after step  1275:  0.0001160
Cost after step  1280:  0.0001044
Cost after step  1285:  0.0001780
Cost after step  1290:  0.0001129
Cost after step  1295:  0.0001243
Cost after step  1300:  0.0001076
Cost after step  1305:  0.0001073
Cost after step  1310:  0.0001409
Cost after step  1315:  0.0000569
Cost after step  1320:  0.0001098
Cost after step  1325:  0.0001571
Cost after step  1330:  0.0001393
Cost after step  1335:  0.0001376
Cost after step  1340:  0.0001057
Cost after step  1345:  0.0001465
Cost after step  1350:  0.0000915
Cost after step  1355:  0.0001298
Cost after ste

Cost after step  2425:  0.0000935
Cost after step  2430:  0.0000754
Cost after step  2435:  0.0000765
Cost after step  2440:  0.0000478
Cost after step  2445:  0.0000867
Cost after step  2450:  0.0000390
Cost after step  2455:  0.0000475
Cost after step  2460:  0.0000602
Cost after step  2465:  0.0000537
Cost after step  2470:  0.0000550
Cost after step  2475:  0.0000474
Cost after step  2480:  0.0000517
Cost after step  2485:  0.0000784
Cost after step  2490:  0.0000400
Cost after step  2495:  0.0000502
Cost after step  2500:  0.0000451
Cost after step  2505:  0.0000297
Cost after step  2510:  0.0000642
Cost after step  2515:  0.0000048
Cost after step  2520:  0.0000356
Cost after step  2525:  0.0000890
Cost after step  2530:  0.0000711
Cost after step  2535:  0.0000723
Cost after step  2540:  0.0000441
Cost after step  2545:  0.0000825
Cost after step  2550:  0.0000357
Cost after step  2555:  0.0000430
Cost after step  2560:  0.0000555
Cost after step  2565:  0.0000505
Cost after ste

Cost after step  3635:  0.0000401
Cost after step  3640:  0.0000179
Cost after step  3645:  0.0000501
Cost after step  3650:  0.0000128
Cost after step  3655:  0.0000128
Cost after step  3660:  0.0000218
Cost after step  3665:  0.0000263
Cost after step  3670:  0.0000216
Cost after step  3675:  0.0000188
Cost after step  3680:  0.0000275
Cost after step  3685:  0.0000345
Cost after step  3690:  0.0000122
Cost after step  3695:  0.0000193
Cost after step  3700:  0.0000186
Cost after step  3705:  0.0000039
Cost after step  3710:  0.0000279
Cost after step  3715:  0.0000025
Cost after step  3720:  0.0000077
Cost after step  3725:  0.0000530
Cost after step  3730:  0.0000365
Cost after step  3735:  0.0000382
Cost after step  3740:  0.0000165
Cost after step  3745:  0.0000482
Cost after step  3750:  0.0000117
Cost after step  3755:  0.0000113
Cost after step  3760:  0.0000200
Cost after step  3765:  0.0000249
Cost after step  3770:  0.0000199
Cost after step  3775:  0.0000174
Cost after ste

Cost after step  4845:  0.0000338
Cost after step  4850:  0.0000040
Cost after step  4855:  0.0000024
Cost after step  4860:  0.0000076
Cost after step  4865:  0.0000143
Cost after step  4870:  0.0000085
Cost after step  4875:  0.0000077
Cost after step  4880:  0.0000164
Cost after step  4885:  0.0000159
Cost after step  4890:  0.0000030
Cost after step  4895:  0.0000073
Cost after step  4900:  0.0000079
Cost after step  4905:  0.0000000
Cost after step  4910:  0.0000135
Cost after step  4915:  0.0000148
Cost after step  4920:  0.0000009
Cost after step  4925:  0.0000370
Cost after step  4930:  0.0000219
Cost after step  4935:  0.0000232
Cost after step  4940:  0.0000065
Cost after step  4945:  0.0000330
Cost after step  4950:  0.0000037
Cost after step  4955:  0.0000020
Cost after step  4960:  0.0000070
Cost after step  4965:  0.0000137
Cost after step  4970:  0.0000079
Cost after step  4975:  0.0000071
Cost after step  4980:  0.0000158
Cost after step  4985:  0.0000150
Cost after ste

In [361]:
print("Optimized rotation angles: {}".format(params))

Optimized rotation angles: [ 0.38813364  1.00791153  0.24022425  0.70250559  0.21662048  0.00376867
 -0.76151783]


# Testing

In [370]:
class_to_prediction = {
    'Setosa': 0.33333,
    'Versicolor': 0.66667,
    'Virginica': 1.00000,
}

index = 51

features = dataset[index]
# expected_value = dataset_class['variety'][index]
value = circuit(features, params)

predicted_class = ''
current_diff = 10000000000000000
for iris_class, prediction_value in class_to_prediction.items():
    class_diff = np.sqrt((prediction_value - value) ** 2)
    if class_diff > current_diff:
        continue

    predicted_class = iris_class
    current_diff = class_diff

print(predicted_class)
print(current_diff)

Versicolor
0.019758703071174
